In [8]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt

# others
import os

In [3]:
def plot_history(H, epochs):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss", linestyle=":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc", linestyle=":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()

In [42]:
# Loading data

# Input folder
data_path = os.path.join("..","data", "Tobacco3482")
dirs = sorted(os.listdir(data_path))
images = []

for directory in dirs:
    subfolder = os.path.join(data_path, directory)
    filenames = sorted(os.listdir(subfolder))

    for image in filenames:
        file_path = os.path.join(subfolder, image)
        if file_path.endswith('.jpg'):
            image_data = load_img(file_path, target_size=(224, 224)) 
            images.append({"label": directory, "image": image_data})

In [48]:
# Convert each image to a numpy array
image_arrays = [img_to_array(image['image']) for image in images]

In [50]:
# Reshaping data for the model using list comprehension
image_reshaped = [image_array.reshape((1, image_array.shape[0], image_array.shape[1], image_array.shape[2])) for image_array in image_arrays]

In [56]:
# Prepare each image for the VGG model
processed_images = [preprocess_input(image_reshape) for image_reshape in image_reshaped]

In [57]:
processed_images

[array([[[[151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          ...,
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ]],
 
         [[151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          ...,
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ]],
 
         [[151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          ...,
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ]],
 
         ...,
 
         [[151.061  , 138.22101, 131.32   ],
          [151.061  , 138.22101, 131.32   ],
          [151.061  , 138.2210